In [110]:
%%writefile gen-isolation.py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from tabulate import tabulate
import re
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import os


bench = {'cut_sgemm-0':0, 'cut_sgemm-1':0, 'cut_wmma-0': 0, 'cut_wmma-1': 0, 
         'parb_stencil-0': 1, 'parb_sgemm-0': 0,
         'parb_lbm-0': 1, 'parb_spmv-0': 1, 'parb_cutcp-0': 0}

mpl.style.use('seaborn-paper')

def avg_array(s):
    result = [np.average(np.array(v[1:-1].split(' ')).astype(float)) for v in s]
    return np.array(result)

def plot_heatmap(df, regex_x, label_x, title, axis, cbar, cbar_ax, cmap, metric_key, metric_name, font):
    df['x'] = df['config'].apply(lambda x: re.search(regex_x, x).group(1)).astype(int)

    regex_l2 = r'PARTITION_L2_0:(.*):[0-9|\.]+'
    df['L2'] = df['config'].apply(lambda x: re.search(regex_l2, x).group(1)).astype(float)
    df.sort_values(['L2', 'x'], inplace=True, ascending=[False, True])

    num_sm = len(df['L2'].unique())
    data = np.split(df[metric_key].values, num_sm)

    if df[metric_key].dtype == np.int64:
        fmt = 'd'
    else:
        fmt = '.4f'

    sns.set(font_scale=font)
    sns.heatmap(data, ax=axis, linewidth=0.2, linecolor='white', 
                square=True, cmap=cmap, #vmin=cbar_lim[0], vmax=cbar_lim[1],
                xticklabels=df['x'].unique(), yticklabels=df['L2'].unique(),
                cbar=cbar, cbar_ax=cbar_ax, annot=True, fmt=fmt,
                cbar_kws={'label': metric_name}
               )
    axis.set_xlabel(label_x)
    axis.set_ylabel('Normalized L2 Partition Size')
    axis.set_title(title, fontsize=16)

def plot_intra_inter(df_intra, df_inter, benchmark, figsize, font, inter_valid=True):
    

    def plot_line(df, regex_legend, legend, title, axis, metric_key, metric_name):
        df['x'] = df['config'].apply(lambda x: re.search(regex_legend, x).group(1)).astype(int)
        df['x_c'] = df['x'].apply(lambda x: '{0} {1}'.format(x, legend)).astype('category')

        regex_l2 = r'PARTITION_L2_0:(.*):[0-9|\.]+'
        df['L2'] = df['config'].apply(lambda x: re.search(regex_l2, x).group(1)).astype(float)       
        
        
        sns.set(font_scale=font)
        sns.lineplot(x='L2', y=metric_key, hue='x_c', data=df, ax=axis)
        
        axis.set_xlabel('L2 Partition Size')
        axis.set_ylabel(metric_name)
        axis.set_title(title)
        axis.xaxis.grid()

    def plot_page_single(pdf, metric_key, metric_name):
        cmap_1 = sns.cubehelix_palette(rot=-.4, dark=0.3, as_cmap=True)

        fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=figsize)   
        fig.suptitle(benchmark + ': ' + metric_name)

        plot_heatmap(df_intra.copy(), regex_x=r'INTRA_0:(.*):0_CTA', 
                     label_x='Concurrent CTAs/SM', title='Intra-SM', axis=ax1, 
                     cbar=True, cbar_ax=None,
                     cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
                     font=font,
                    ) 

        plot_line(df_intra.copy(), regex_legend=r'INTRA_0:(.*):0_CTA', 
                 legend='Concurrent CTAs/SM', title='Intra-SM', axis=ax2, 
                 metric_key=metric_key, metric_name=metric_name,
                 )

        pdf.savefig(fig)
        plt.close()
       
    def plot_page_both(pdf, metric_key, metric_name):

        cmap_1 = sns.cubehelix_palette(rot=.1, as_cmap=True)
        cmap_2 = sns.cubehelix_palette(rot=-.4, dark=0.3, as_cmap=True)

        # ignore bypass L2D for now
        def mask(df):
            return df['config'].str.contains('BYPASS_L2D')

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=figsize)   
        fig.suptitle(benchmark + ': ' + metric_name)


        plot_heatmap(df_intra[~mask(df_intra)].copy(), regex_x=r'INTRA_0:(.*):0_CTA', 
                     label_x='Concurrent CTAs/SM', title='Intra-SM', axis=ax1, 
                     cbar=True, cbar_ax=None,
                     cmap=cmap_1, metric_key=metric_key, metric_name=metric_name, font=font,
                    ) 
        plot_heatmap(df_inter[~mask(df_inter)].copy(), regex_x=r'INTER_0:(.*):0_SM', 
                     label_x='# of SM', title='Inter-SM', axis=ax2,
                     cbar=True, cbar_ax=None,
                     cmap=cmap_2, metric_key=metric_key, metric_name=metric_name, font=font,
                    ) 
        plot_line(df_intra[~mask(df_intra)].copy(), regex_legend=r'INTRA_0:(.*):0_CTA', 
                 legend='Concurrent CTAs/SM', title='Intra-SM', axis=ax3, 
                 metric_key=metric_key, metric_name=metric_name,
                 )
        plot_line(df_inter[~mask(df_inter)].copy(), regex_legend=r'INTER_0:(.*):0_SM', 
                 legend='# of SM', title='Inter-SM', axis=ax4, 
                 metric_key=metric_key, metric_name=metric_name,
                 )

        pdf.savefig(fig)
        plt.close()

    def plot_page(pdf, metric_key, metric_name):
        if inter_valid:
            plot_page_both(pdf, metric_key, metric_name)
        else:
            plot_page_single(pdf, metric_key, metric_name)

    # scale all IPC to baseline
    baseline = df_seq[df_seq['pair_str'] == benchmark]['ipc'].values[0]
    df_intra['norm_ipc'] = df_intra['ipc'] / baseline
    
    df_intra['avg_dram_bw'] = df_intra['dram_bw'].transform(avg_array)
    
    # calculate DRAM idleness
    df_intra['dram_idleness'] = np.divide(df_intra['mem_idle'].transform(avg_array),
                                          df_intra['total_cmd'].transform(avg_array))
    
    if inter_valid:
        df_inter['norm_ipc'] = df_inter['ipc'] / baseline
        df_inter['avg_dram_bw'] = df_inter['dram_bw'].transform(avg_array)
    
        # calculate DRAM idleness
        df_inter['dram_idleness'] = np.divide(df_inter['mem_idle'].transform(avg_array),
                                              df_inter['total_cmd'].transform(avg_array))
 
    filename = '{0}-{1}.pdf'.format(benchmark, 'both' if inter_valid else 'intra')
    filename = os.path.join('plots', filename)
    with PdfPages(filename) as pdf:
        plot_page(pdf, 'norm_ipc', 'Normalized IPC')
        plot_page(pdf, 'avg_dram_bw', 'DRAM Bandwidth Utilization')
        plot_page(pdf, 'dram_idleness', 'DRAM IDLE RATE')
        plot_page(pdf, 'l2_miss_rate', 'L2 Miss Rate')
        plot_page(pdf, 'l2_BW', 'L2 Bandwidth')
        plot_page(pdf, 'l2_total_accesses', 'L2 Total Accesses')
        plot_page(pdf, 'l1D_miss_rate', 'L1 Miss Rate')
        plot_page(pdf, 'avg_mem_lat', 'Average Memory Latency')
        
    return df_intra


Writing gen-isolation.py


In [69]:
df_seq = pd.read_csv('seq.csv')
df_intra = pd.read_csv('intra.csv')
df_inter = pd.read_csv('inter.csv')
print(df_inter.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'dram_eff', 'dram_bw', 'row_buffer_locality',
       'mrqq', 'total_cmd', 'wasted_col', 'wasted_row', 'mem_idle', 'CCDLc',
       'WTRc', 'RTWc', 'RCDc', 'RCDWRc'],
      dtype='object')


In [75]:
df_stencil_intra = df_intra[df_intra['pair_str']=='parb_stencil-0'].copy()
df_stencil_inter = df_inter[df_inter['pair_str']=='parb_stencil-0'].copy()
plot_intra_inter(df_stencil_intra, df_stencil_inter, 'parb_stencil-0', figsize=(30, 30), font=1.8)

In [76]:
df_cutcp_intra = df_intra[df_intra['pair_str']=='parb_cutcp-0'].copy()
df_cutcp_inter = df_inter[df_inter['pair_str']=='parb_cutcp-0'].copy()
plot_intra_inter(df_cutcp_intra, df_cutcp_inter, 'parb_cutcp-0', figsize=(30,30), font=1.8)

## Intra SM Only

In [92]:
def gen_intra_only(bench):
    df_bench = df_intra[df_intra['pair_str']==bench].copy()
    plot_intra_inter(df_bench, None, bench, figsize=(30, 10), font=1.8, inter_valid=False)
    return df_bench

In [93]:
df_sgemm0 = gen_intra_only('cut_sgemm-0')
df_sgemm1 = gen_intra_only('cut_sgemm-1')
df_wmma0 = gen_intra_only('cut_wmma-0')
df_wmma1 = gen_intra_only('cut_wmma-1')

In [94]:
df_cutcp = gen_intra_only('parb_cutcp-0')
df_stencil = gen_intra_only('parb_stencil-0')
df_lbm = gen_intra_only('parb_lbm-0')
df_spmv = gen_intra_only('parb_spmv-0')

In [109]:
fig_tot, ((ax1_tot, ax2_tot, ax3_tot, ax4_tot), (ax5_tot, ax6_tot, ax7_tot, ax8_tot)) \
    = plt.subplots(2, 4, figsize=(40, 30))   

cmap_1 = sns.cubehelix_palette(rot=-.4, dark=0.3, as_cmap=True)
metric_key = 'norm_ipc'
metric_name = 'Normalized IPC'

plot_heatmap(df_sgemm0, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='cut_sgemm-0', axis=ax1_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)
plot_heatmap(df_sgemm1, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='cut_sgemm-1', axis=ax2_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)
plot_heatmap(df_wmma0, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='cut_wmma-0', axis=ax3_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)
plot_heatmap(df_wmma1, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='cut_wmma-1', axis=ax4_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)

plot_heatmap(df_cutcp, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='cutcp', axis=ax5_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)
plot_heatmap(df_stencil, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='stencil', axis=ax6_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)
plot_heatmap(df_lbm, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='lbm', axis=ax7_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)
plot_heatmap(df_spmv, regex_x=r'INTRA_0:(.*):0_CTA', 
             label_x='Concurrent CTAs/SM', title='spmv', axis=ax8_tot, 
             cbar=True, cbar_ax=None,
             cmap=cmap_1, metric_key=metric_key, metric_name=metric_name,
             font=1.4,)

fig_tot.savefig('plots/total.pdf')
plt.close()




In [118]:
def keyfunc(**kwargs):
    print(kwargs)
    print(hello)

In [119]:
keyfunc(hello=1)

{'hello': 1}


NameError: name 'hello' is not defined